<a href="https://colab.research.google.com/github/amaar-dev/Face_mask_DT/blob/main/Face_mask_DT_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training model to recognise face masks

## Importing Data set to train Model

In [3]:
from pathlib import Path
import kagglehub
import os


path = kagglehub.dataset_download("vijaykumar1799/face-mask-detection")

print("Path to dataset files:", path)

dataset_dir = Path(path) / "Dataset"
with_mask_dir = dataset_dir / "with_mask"
without_mask_dir = dataset_dir / "without_mask"


#check folder
print("Files in dataset folder: ")
print(os.listdir(path))


print("with_mask : ", len(os.listdir(with_mask_dir)), "images")
print("without_mask : ", len(os.listdir(without_mask_dir)), "images")





Path to dataset files: /kaggle/input/face-mask-detection
Files in dataset folder: 
['Dataset']
with_mask :  2994 images
without_mask :  2994 images


##Training Model MobileNetV2


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


### Initialize Data


In [17]:
dataset_path = str(dataset_dir)

data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = data_gen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    classes=['with_mask', 'without_mask']
)
val_data = data_gen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    classes=['with_mask', 'without_mask']
)

Found 4792 images belonging to 2 classes.
Found 1196 images belonging to 2 classes.


###Adding custom layers to mobilenetv2


In [9]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
# stop top layers

#adding custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x) #2 classes

Face_Mask_DT_model = Model(inputs=base_model.input, outputs=predictions)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [19]:
Face_Mask_DT_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

###Training


In [20]:
history = Face_Mask_DT_model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9190 - loss: 0.2102

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


150/150 ━━━━━━━━━━━━━━━━━━━━ 355s 2s/step - accuracy: 0.9192 - loss: 0.2097 - val_accuracy: 0.9691 - val_loss: 0.1006
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 333s 2s/step - accuracy: 0.9724 - loss: 0.0812 - val_accuracy: 0.9699 - val_loss: 0.0810
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 348s 2s/step - accuracy: 0.9795 - loss: 0.0573 - val_accuracy: 0.9682 - val_loss: 0.0787
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 338s 2s/step - accuracy: 0.9735 - loss: 0.0572 - val_accuracy: 0.9699 - val_loss: 0.0840
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.9834 - loss: 0.0453 - val_accuracy: 0.9724 - val_loss: 0.0681
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 331s 2s/step - accuracy: 0.9815 - loss: 0.0446 - val_accuracy: 0.9674 - val_loss: 0.0720
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.9833 - loss: 0.0417 - val_accuracy: 0.9791 - val_loss: 0.0520
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.9835 - loss: 0.0414 - val_accuracy: 0.975

In [22]:

Face_Mask_DT_model.save("Face_Mask_DT_Model.h5")
